In [1]:
import os
import time
import Turbine
import mat_properties as prop
import numpy as n
import pandas as pd

os.environ["RPPREFIX"] = r"C:/Program Files (x86)/REFPROP"

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)


# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)

water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,492.20,5.1990,3414.093682,41.97,NaN,NaN
ENDOFVD,210.00,0.5490,2874.837567,41.97,NaN,NaN
DROSND-TURBND,199.40,0.4000,2859.669112,10.57,NaN,NaN
SMESHEND,78.16,0.4000,327.573430,52.54,NaN,"0,02323321"
DOOTB2,131.40,0.2542,2724.861533,52.54,NaN,NaN
DOOTB1,NaN,0.1525,2686.016192,52.54,NaN,NaN
INCND,NaN,0.1525,2686.016192,52.54,NaN,NaN
INKOND,NaN,0.0017,2341.001493,52.54,NaN,NaN


In [2]:
calctolerance = 10**-3
calcmethod = "hybr"
KPD_SP = 0.99  # эффективность сетевых подогревателей
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)

Turb = Turbine.turbine(
    water,
    "DROSVD-TURBVD",
    "ENDOFVD",
    "DROSND-TURBND",
    "SMESHEND",
    "DOOTB2",
    "DOOTB1",
    "INCND",
    "INKOND",
    water_streams0,
    water_streams,
)

start_time = time.time()

res = Turb.calculate(diafragma=0, maxiterations=50, calctolerance=10**-5)

print("fin:--- %s сек. ---" % round((time.time() - start_time), 1))
res

fin:--- 0.1 сек. ---


In [3]:
# надо посчитать потерю давления НД (готово)
# отборы неправильно (второй отбор завышен, надопроверить исходные данные функции)
water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,492.131678,5.185478,3414.093682,41.97,6.935224,100
ENDOFVD,189.568854,0.367788,2840.733518,41.97,7.167066,100
DROSND-TURBND,199.389973,0.399584,2859.669112,10.57,7.170110,100
SMESHEND,191.385057,0.367788,2844.542982,52.54,7.175282,100
DOOTB2,125.345444,0.169832,2720.464941,52.54,7.236685,100
DOOTB1,99.990786,0.101385,2648.119613,52.54,7.280703,98.784108
INCND,99.990786,0.101385,2648.119613,52.54,7.280703,98.784108
INKOND,14.947194,0.001700,2242.770527,52.54,7.790622,88.421451


In [4]:
water_streams0['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,Unnamed: 6,Unnamed: 7
DROSVD-TURBVD,508.600000,7.8450,3422.525472,63.40,NaN,NaN,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN,NaN,NaN
DROSND-TURBND,209.800000,0.6110,2871.426945,14.76,7.003739,NaN,NaN
SMESHEND,197.800000,0.5490,2848.521603,78.16,7.003601,NaN,NaN
DOOTB2,131.400000,0.2542,2724.861533,78.16,7.065770,NaN,NaN
DOOTB1,NaN,0.1525,2652.000000,78.16,7.109000,NaN,0.023256
INCND,NaN,0.1525,2652.000000,78.16,NaN,NaN,NaN
INKOND,NaN,0.0080,2338.024560,78.16,NaN,NaN,0.000064


In [5]:
Get_value='P'
Error=(water_streams['DROSVD-TURBVD':'INKOND'][Get_value]-water_streams0['DROSVD-TURBVD':'INKOND'][Get_value])/water_streams0['DROSVD-TURBVD':'INKOND'][Get_value]*100
Error

DROSVD-TURBVD   -33.900853
ENDOFVD         -33.007654
DROSND-TURBND   -34.601683
SMESHEND        -33.007654
DOOTB2          -33.189594
DOOTB1          -33.518260
INCND           -33.518260
INKOND          -78.750000
Name: P, dtype: float64

In [6]:
Turb.calculate_power()

{'Ni': 55.68105086827658}